In [112]:
import numpy as np
import random
import math
import time
from shapely.geometry import Polygon, MultiPolygon, Point, box
from shapely import wkt
import folium
from folium.features import DivIcon
from IPython.display import display, clear_output
from branca.element import Figure

In [5]:
def generate_random_polygon(bbox, num_points):
    """Generate a random polygon within the bounding box with the given number of points."""
    min_x, min_y, max_x, max_y = bbox
    width = max_x - min_x
    height = max_y - min_y
    
    # Generate random center point
    center_x = min_x + width / 2
    center_y = min_y + height / 2
    
    # Generate random points at varied distances around the center
    radius = min(width, height) / 2 * 0.8  # 80% of half the smallest dimension
    
    # Generate points around the center at varying distances
    points = []
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        # Vary the radius a bit
        r = radius * random.uniform(0.5, 1.0)
        x = center_x + r * math.cos(angle)
        y = center_y + r * math.sin(angle)
        points.append((x, y))
    
    # Close the polygon
    points.append(points[0])
    return points

def genCase1(bbox, num_points):
    """Case 1: Simple polygon with 3 or more points."""
    polygon = generate_random_polygon(bbox, num_points)
    return f"POLYGON (({' ,'.join([f'{x} {y}' for x, y in polygon])}))"

In [ ]:
def create_polygon_with_area_percentage(bbox, percentage):
    """Create a polygon with a specific percentage of the bounding box area."""
    min_x, min_y, max_x, max_y = bbox
    width = max_x - min_x
    height = max_y - min_y
    
    # Calculate the scaling factor to achieve the desired area percentage
    scale_factor = math.sqrt(percentage)
    
    # Calculate the dimensions of the new polygon
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    # Calculate the center of the bounding box
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    # Calculate the coordinates of the new polygon
    new_min_x = center_x - new_width / 2
    new_min_y = center_y - new_height / 2
    new_max_x = center_x + new_width / 2
    new_max_y = center_y + new_height / 2
    
    # Create the new polygon
    return [(new_min_x, new_min_y), (new_min_x, new_max_y), 
            (new_max_x, new_max_y), (new_max_x, new_min_y), (new_min_x, new_min_y)]

def genCase2(bbox, area_percentage=0.5):
    """Case 2: Simple polygon with varying area percentage of BBOX."""
    polygon = create_polygon_with_area_percentage(bbox, area_percentage)
    return f"POLYGON (({' ,'.join([f'{x} {y}' for x, y in polygon])}))"

In [42]:
def genPolygon_AreaFIXED(INPUT_BBOX, num_points=None, area_percentage=None):
    """
    Generate different polygons within the INPUT_BBOX in EPSG:4326 - WGS 84
    Ranges from small area polygon to large polygons with large area in OGC WKT format
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the polygon should occupy. If None, a random value is used.
        
    Returns:
        String: WKT format polygon
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
       
    # Get bbox dimensions
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat
    
    # Calculate center of bbox
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Calculate anywhere of bbox
    # center_lon = random.uniform(min_lon, max_lon)
    # center_lat = random.uniform(min_lat, max_lat)
    
    # Calculate polygon dimensions based on area percentage
    # For simplicity, we'll create a rectangular polygon
    scale_factor = math.sqrt(area_percentage)
    poly_width = bbox_width * scale_factor
    poly_height = bbox_height * scale_factor
    
    # Calculate polygon corners
    half_width = poly_width / 2
    half_height = poly_height / 2
    
    # Create a more interesting polygon by adding some random points and perturbations
    points = []
    
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Base radius is determined by the desired area
        x_radius = half_width
        y_radius = half_height
        
        # Add some randomness but maintain approximate area
        radius_factor = random.uniform(0.9, 1.1)
        
        x = center_lon + math.cos(angle) * x_radius * radius_factor
        y = center_lat + math.sin(angle) * y_radius * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        points.append((x, y))
    
    # Close the polygon by adding the first point at the end
    points.append(points[0])
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in points])
    wkt += "))"

    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
    return wkt

In [214]:
def view_polygons_on_osm(i, polygon, bbox, area_percentage=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#{(i * 50) % 256:02x}0000': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north+0.2, east+0.5],
        icon=DivIcon(
            icon_size=(350,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 20pt;">POINTS : '+str(i)+' | AREA :'+str(area_percentage)+'</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # Final display to keep the map visible after the loop
    # display(m)
    return m

In [215]:
## USER GLOBAL VARIABLES
# INPUT_BBOX = [11.015629, 55.128649, 24.199222, 69.380313] ## Sweden
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 10  # 1,2 points are not polygon, so start with atleast 3 points 
NUM_OF_HOLES = 100
PERCENTAGE_OF_AREA = 0.05

In [216]:
for i in range(3, NUM_OF_POINTS):
    polygon = genCase1(INPUT_BBOX, i)
    print(f"Points: {i} - {polygon}")
    view_polygons_on_osm(i, polygon, INPUT_BBOX)

Points: 9 - POLYGON ((11.591948575407438 48.15486111114964 ,11.590123086213362 48.19528779168082 ,11.548529636228329 48.19220758953993 ,11.508083173194201 48.213510553379926 ,11.49251373705412 48.172852417301804 ,11.498662257383527 48.139107683382164 ,11.52306050056798 48.12215316089234 ,11.551491504224913 48.10071704463822 ,11.59798103942076 48.10784082498033 ,11.591948575407438 48.15486111114964))


## BM 1 - POLYGON WITH FIXED AREA and VARYING NUMBER OF POINTS

In [223]:
## USER GLOBAL VARIABLES
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 10  # FIXED
PERCENTAGE_OF_AREA = 0.5 # VARYING

In [225]:
for i in range(3, NUM_OF_POINTS):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, i, area_percentage=PERCENTAGE_OF_AREA)
    print(f"Points: {i} - {polygon}")
    view_polygons_on_osm(i, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Points: 9 - POLYGON ((11.66879957849277 48.15486111114964, 11.646378718744508 48.19998591683446, 11.56426631098352 48.22004953750841, 11.47637383749038 48.21334396975782, 11.432252172609942 48.175420048393676, 11.427403159153409 48.133393353606124, 11.47959895284965 48.099254754289525, 11.56341861165174 48.09214829252692, 11.636346053682523 48.114071300508336, 11.66879957849277 48.15486111114964))


## BM 2 - POLYGON WITH FIXED POINTS and VARYING AREA

In [219]:
## USER GLOBAL VARIABLES
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305]
NUM_OF_POINTS = 4  # FIXED
# PERCENTAGE_OF_AREA = 0.05 # VARYING

In [220]:
for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.02):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(f"Area: {PERCENTAGE_OF_AREA*100.0:.2f}% - {polygon}")
    view_polygons_on_osm(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Area: 100.00% - POLYGON ((11.723194444453823 48.15486111114964, 11.541944444453677 48.24819444448305, 11.360694444453532 48.15486111114964, 11.541944444453677 48.06152777781623, 11.723194444453823 48.15486111114964))


## BM 3 - POLYGON WITH VARYING POINTS and VARYING AREA